In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import get_file
import nb_utils
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from keras.preprocessing.text import Tokenizer
from keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from itertools import chain

In [2]:
emotion_df = pd.read_csv('./text_emotion.csv')
emotion_df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [3]:
emotion_df['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [4]:
VOCAB_SIZE = 50000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)

In [5]:
def create_lstm_model(vocab_size, embedding_size=None, embedding_weights=None):
    message = layers.Input(shape=(None,), dtype='int32', name='title')
    embedding = make_embedding('message_vec', vocab_size, embedding_size, embedding_weights)(message)

    lstm_1 = layers.LSTM(units=128, return_sequences=False)(embedding)
#     lstm_2 = layers.LSTM(units=128, return_sequences=False)(lstm_1)
    category = layers.Dense(units=len(label_encoder.classes_), activation='softmax')(lstm_1)
    
    model = models.Model(
        inputs=[message],
        outputs=[category],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [6]:
w2v, idf = nb_utils.load_w2v(tokenizer)

In [7]:
def make_embedding(name, vocab_size, embedding_size, weights=None, mask_zero=True):
    if weights is not None:
        return layers.Embedding(mask_zero=mask_zero, input_dim=vocab_size, 
                                output_dim=weights.shape[1], 
                                weights=[weights], trainable=False, 
                                name='%s/embedding' % name)
    else:
        return layers.Embedding(mask_zero=mask_zero, input_dim=vocab_size, 
                                output_dim=embedding_size,
                                name='%s/embedding' % name)

In [8]:
label_encoder = LabelEncoder()

In [9]:
y = label_encoder.fit_transform(emotion_df['sentiment'])

In [10]:
chars = list(sorted(set(chain(*emotion_df['content']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in emotion_df['content'])
char_vectors = []
for txt in emotion_df['content']:
    vec = np.zeros((max_sequence_len, len(char_to_idx)))
    vec[np.arange(len(txt)), [char_to_idx[ch] for ch in txt]] = 1
    char_vectors.append(vec)
char_vectors = np.asarray(char_vectors)
char_vectors = pad_sequences(char_vectors)
labels = label_encoder.transform(emotion_df['sentiment'])

In [11]:
training_count = int(0.9 * len(char_vectors))

In [12]:
tokens = tokenizer.texts_to_sequences(emotion_df['content'])
tokens = pad_sequences(tokens)
training_tokens, training_labels = tokens[:training_count], labels[:training_count]

In [13]:
lstm_model = create_lstm_model(VOCAB_SIZE, embedding_weights=w2v)
lstm_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
title (InputLayer)           [(None, None)]            0         
_________________________________________________________________
message_vec/embedding (Embed (None, None, 300)         15000000  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 13)                1677      
Total params: 15,221,325
Trainable params: 221,325
Non-trainable params: 15,000,000
_________________________________________________________________


In [57]:
lstm_model.fit(training_tokens, training_labels, epochs=10, batch_size=128)

Epoch 1/10
36000/36000 [==============================] - 22s - loss: 2.0147 - acc: 0.3125    
Epoch 2/10
36000/36000 [==============================] - 19s - loss: 1.8909 - acc: 0.3542    
Epoch 3/10
36000/36000 [==============================] - 19s - loss: 1.8447 - acc: 0.3700    
Epoch 4/10
36000/36000 [==============================] - 19s - loss: 1.8151 - acc: 0.3787    
Epoch 5/10
36000/36000 [==============================] - 19s - loss: 1.7889 - acc: 0.3860    
Epoch 6/10
36000/36000 [==============================] - 19s - loss: 1.7636 - acc: 0.3927    
Epoch 7/10
36000/36000 [==============================] - 19s - loss: 1.7409 - acc: 0.3996    
Epoch 8/10
36000/36000 [==============================] - 19s - loss: 1.7165 - acc: 0.4071    
Epoch 9/10
36000/36000 [==============================] - 19s - loss: 1.6898 - acc: 0.4179    
Epoch 10/10
36000/36000 [==============================] - 19s - loss: 1.6584 - acc: 0.4284    


In [58]:
lstm_model.evaluate(test_tokens, test_labels)

3936/4000 [============================>.] - ETA: 0s

[1.918457947731018, 0.36149999999999999]